In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
nu = 0.01
x_max, y_max = 1.0, 1.0

# Define the neural network
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.net = nn.Sequential()
        for i in range(len(layers) - 1):
            self.net.add_module(f"layer_{i}", nn.Linear(layers[i], layers[i+1]))
            if i != len(layers) - 2:
                self.net.add_module(f"tanh_{i}", nn.Tanh())

    def forward(self, x):
        return self.net(x)

# Define PDE residual
def pde_residual(model, x, y, t):
    X = torch.cat([x, y, t], dim=1).to(device)
    X.requires_grad_(True)
    u = model(X)

    grads = autograd.grad(u, X, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = grads[:, 0:1]
    u_y = grads[:, 1:2]
    u_t = grads[:, 2:3]

    u_xx = autograd.grad(u_x, X, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
    u_yy = autograd.grad(u_y, X, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]

    return u_t - nu * (u_xx + u_yy)

# Sample training points
def generate_points(n_points):
    x = torch.rand((n_points, 1)) * x_max
    y = torch.rand((n_points, 1)) * y_max
    t = torch.rand((n_points, 1))
    return x.to(device), y.to(device), t.to(device)

# Boundary and initial condition points
def boundary_points(n_points):
    t = torch.rand((n_points, 1))
    x0 = torch.zeros_like(t)
    x1 = torch.ones_like(t) * x_max
    y0 = torch.zeros_like(t)
    y1 = torch.ones_like(t) * y_max

    # BC1: u(t, 0, y) = 0
    y_rand = torch.rand_like(t) * y_max
    bc1 = (x0, y_rand, t, torch.zeros_like(t))

    # BC2: u(t, xmax, y) = 0
    bc2 = (x1, y_rand, t, torch.zeros_like(t))

    # BC3: u(t, x, 0) = 0
    x_rand = torch.rand_like(t) * x_max
    bc3 = (x_rand, y0, t, torch.zeros_like(t))

    # BC4: u(t, x, ymax) = 1
    bc4 = (x_rand, y1, t, torch.ones_like(t))

    return [bc1, bc2, bc3, bc4]

def initial_condition(n_points):
    x = torch.rand((n_points, 1)) * x_max
    y = torch.rand((n_points, 1)) * y_max
    t0 = torch.zeros_like(x)
    u0 = torch.zeros_like(x)
    return x.to(device), y.to(device), t0.to(device), u0.to(device)

# Training loop
def train(model, optimizer, epochs=5000, n_train=10000):
    for epoch in range(epochs):
        optimizer.zero_grad()

        # PDE residual loss
        x, y, t = generate_points(n_train)
        res = pde_residual(model, x, y, t)
        loss_pde = torch.mean(res**2)

        # Initial condition loss
        x0, y0, t0, u0 = initial_condition(n_train // 4)
        u_pred0 = model(torch.cat([x0, y0, t0], dim=1))
        loss_ic = nn.MSELoss()(u_pred0, u0)

        # Boundary condition loss
        loss_bc = 0
        for xb, yb, tb, ub in boundary_points(n_train // 4):
            u_pred = model(torch.cat([xb.to(device), yb.to(device), tb.to(device)], dim=1))
            loss_bc += nn.MSELoss()(u_pred, ub.to(device))

        loss_total = loss_pde + loss_ic + loss_bc
        loss_total.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss_total.item():.6f}, PDE: {loss_pde.item():.6f}, IC: {loss_ic.item():.6f}, BC: {loss_bc.item():.6f}")

# Initialize model and optimizer
layers = [3, 50, 50, 50, 1]  # Input: x, y, t | Output: u
model = PINN(layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model
train(model, optimizer, epochs=5000)


Epoch 0, Loss: 1.299810, PDE: 0.001417, IC: 0.006766, BC: 1.291628
Epoch 500, Loss: 0.084843, PDE: 0.006929, IC: 0.020612, BC: 0.057302
Epoch 1000, Loss: 0.066283, PDE: 0.003854, IC: 0.012002, BC: 0.050427
Epoch 1500, Loss: 0.052661, PDE: 0.003137, IC: 0.007869, BC: 0.041655
Epoch 2000, Loss: 0.046945, PDE: 0.002950, IC: 0.007796, BC: 0.036199
Epoch 2500, Loss: 0.040886, PDE: 0.003309, IC: 0.006003, BC: 0.031573
Epoch 3000, Loss: 0.042283, PDE: 0.003635, IC: 0.006060, BC: 0.032588
Epoch 3500, Loss: 0.032624, PDE: 0.003162, IC: 0.004995, BC: 0.024467
Epoch 4000, Loss: 0.033520, PDE: 0.003343, IC: 0.004557, BC: 0.025620
Epoch 4500, Loss: 0.031506, PDE: 0.003404, IC: 0.004185, BC: 0.023917
